# Setup Omero

This notebook goes through checking if Omero can connect to this project through Python. You will need the hostname, your username and password to access it.

In [10]:
# Function to connect to the Omero server
# Might clean this up into a separate py file if more is added to this notebook

from omero.gateway import BlitzGateway
from getpass import getpass

# Function that connects the user to the omero server
def connect(hostname, username, password):
    """
    Connect to an OMERO server
    :param hostname: Host name
    :param username: User
    :param password: Password
    :return: Connected BlitzGateway
    """
    conn = BlitzGateway(username, password,
                        host=hostname, secure=True)
    conn.connect()
    conn.c.enableKeepAlive(60)
    return conn

# Using the function from above
HOST = 'ome2.hpc.sussex.ac.uk' #change if different
conn = connect(HOST, input("Username: "), getpass("OMERO Password: "))
try:
    print('Connected as', conn.getUser().getName())
except AttributeError:
    print('You inputted a wrong username and/or password.')

Connected as rz200


In [11]:
# Get a plate's images
plate_id = 812
plate = conn.getObject("Plate", plate_id)
plate_name=plate.name
print (plate_name)

Rehan_TrainingData_01


In [16]:
import time

In [44]:
# Retrieve all images in a plate
# Should change this to a numpy array
# Lets time this, then change to a list comprehension and time it, then to a numpy array and time it
import matplotlib.pyplot as plt
st = time.time()

dapi_imgs = [] #useless but just to have something to store
wells = list(plate.listChildren())
for i in range(len(wells)):
    print(str(i+1)+'/'+str(len(wells)))
    well = wells[i]
    for number, item in enumerate(well.listChildren()):
        image = well.getImage(number)
        pixels = image.getPrimaryPixels() #what are primary pixels?
        dapi_imgs.append(pixels.getPlane(0,0,0)) #does this get the dapi intensity of cells channel? interesting
        #print(img_dapi.shape)

et = time.time()
print('Execution time:', str(et - st), 'seconds')

1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
20/20
Execution time: 38.15134906768799 seconds


In [200]:
st = time.time()

dapi_imgs = [] #useless but just to have something to store
wells = list(plate.listChildren())
for i in range(len(wells)):
    print(str(i+1)+'/'+str(len(wells)))
    well = wells[i]
    dapi_imgs.extend([well.getImage(number).getPrimaryPixels().getPlane(0,0,0) for number, item in enumerate(well.listChildren())])

et = time.time()
print('Execution time:', str(et - st), 'seconds')

1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
20/20
Execution time: 37.3217396736145 seconds


In [201]:
!pip install omero-cli-zarr

  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5050 sha256=d45c8b58ec57d4476055f723ad6bf426b5053a5280fd771ef254ebbbc89183dc
  Stored in directory: c:\users\rz200\appdata\local\pip\cache\wheels\a3\d7\75\19cd0d2a893cad4bb0b2b16dd572ad2916d19c0d5ee9612511
Successfully built asciitree


In [45]:
st = time.time()
dapi_imgs = [well.getImage(number).getPrimaryPixels().getPlane(0,0,0) for number, item in enumerate(well.listChildren()) for well in wells]
et = time.time()
print('Execution time:', str(et - st), 'seconds')

Execution time: 36.19126009941101 seconds


In [60]:
print(len(wells))
for number, item in enumerate(well.listChildren()):
    print(number)
print(enumerate(well.listChildren())[0])
#print(enumerate(wells[i]))

20
0
1
2
3
4
5
6
7
8


TypeError: 'enumerate' object is not subscriptable

In [63]:
print(len(list(well.listChildren())))

9


In [104]:
import numpy as np
# get all the well objects
wells = np.array(list(plate.listChildren()))
print(wells)
print(type(wells[0]))
#img_index_per_well = np.array([[wells[i].getImage(number).getPrimaryPixels().getPlane(0,0,0) for number, item in enumerate(wells[i].listChildren())] for i in range(len(wells))])

#let's create one numpy array with all images from one well
well_1 = np.arange(9)
print(well_1)
x = np.arange(9)
well_1[x] = wells[0].getImage(x).getPrimaryPixels().getPlane(0,0,0)
print(well_1)

#img_index_per_well[arr > 255] = x

#print(img_index_per_well)
#well = wells
# get the number of all images in a well
#rand_arr_test = np.arange(4)[:,np.newaxis]
#print(rand_arr_test)

[<_WellWrapper id=6450> <_WellWrapper id=6449> <_WellWrapper id=6448>
 <_WellWrapper id=6447> <_WellWrapper id=6446> <_WellWrapper id=6445>
 <_WellWrapper id=6444> <_WellWrapper id=6443> <_WellWrapper id=6442>
 <_WellWrapper id=6441> <_WellWrapper id=6440> <_WellWrapper id=6439>
 <_WellWrapper id=6438> <_WellWrapper id=6437> <_WellWrapper id=6436>
 <_WellWrapper id=6435> <_WellWrapper id=6434> <_WellWrapper id=6433>
 <_WellWrapper id=6432> <_WellWrapper id=6431>]
<class 'omero.gateway._WellWrapper'>
[0 1 2 3 4 5 6 7 8]


TypeError: only size-1 arrays can be converted to Python scalars

In [117]:
x = range(3)
M = len(x)
sample = np.empty((M, 41632))
for i in range(M):
    sample[i, :] = 15
print(sample)

[[15. 15. 15. ... 15. 15. 15.]
 [15. 15. 15. ... 15. 15. 15.]
 [15. 15. 15. ... 15. 15. 15.]]


In [123]:
print(type(wells[0]))

<class 'omero.gateway._WellWrapper'>


In [145]:
x = range(len(np.array(list(plate.listChildren()))))
M = len(x)
sample = np.empty((M, len(list(wells[0].listChildren()))))
y = np.arange(9)
for i in range(M):
    #sample[i, :] = wells[i].getImage(y).getPrimaryPixels().getPlane(0,0,0)
    sample[i,:] = y
print(sample.shape)
print(sample)

(20, 9)
[[0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]]


In [151]:
x = range(len(np.array(list(plate.listChildren()))))
M = len(x)
sample = np.empty((M, len(list(wells[0].listChildren()))))
y = np.arange(9)
for i in range(M):
    #sample[i, :] = wells[i].getImage(y).getPrimaryPixels().getPlane(0,0,0)
    #sample[i,:] = y
def get_DAPI_img(x):
    print(x)
    return wells[0].getImage(x).getPrimaryPixels().getPlane(0,0,0)
np.apply_along_axis(get_DAPI_img, axis=1, arr=sample)
print(sample.shape)
print(sample)

[0.00000000e+000 6.95207474e-310 6.77364000e-321             nan
 1.12646967e-321 0.00000000e+000 2.11160529e+257 6.61020367e-062
 3.63513182e+228]


TypeError: only size-1 arrays can be converted to Python scalars

In [178]:
import numpy as np
wells = np.array(list(plate.listChildren()))
x = range(len(np.array(list(plate.listChildren()))))
M = len(x)
sample = np.zeros((M,len(list(wells[0].listChildren()))))
y = np.arange(9)
for i in range(M):
    #sample[i, :] = wells[i].getImage(y).getPrimaryPixels().getPlane(0,0,0)
    sample[i,:] = y
print(sample)
for i in range(len(sample)):
    for v in sample[i,:]:
        #print(type(int(v)))
        sample[i,int(v)] = np.array(wells[0].getImage(v).getPrimaryPixels().getPlane(0,0,0))
print(sample)

[[0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]]


ValueError: setting an array element with a sequence.

In [198]:
st = time.time()

sample = np.arange(9)
sample[:] = wells[0].getImage(1).getPrimaryPixels().getPlane(0,0,0)
#for i in range(9):
    #sample[i] = wells[0].getImage(i).getPrimaryPixels().getPlane(0,0,0)

et = time.time()
print('Execution time:', str(et - st), 'seconds')


print(sample)


Execution time: 0.27227306365966797 seconds
[[3480 3930 4382 ...  418  423  374]
 [2764 3295 3656 ...  382  395  398]
 [2084 2853 2965 ...  448  418  389]
 ...
 [ 442  552  443 ...  570  606  592]
 [ 538  530  492 ...  594  620  695]
 [ 595  544  511 ...  548  587  670]]


In [181]:
type(wells[0].getImage(0).getPrimaryPixels().getPlane(0,0,0))

numpy.ndarray

In [197]:
mymatrix = np.matrix([[11,12,13],
                      [21,22,23],
                      [31,32,33]])
def myfunction(x):
    print('x',x)
    print()
    return x+1

print(myfunction(mymatrix))

x [[11 12 13]
 [21 22 23]
 [31 32 33]]

[[12 13 14]
 [22 23 24]
 [32 33 34]]


In [139]:
prices = np.full(100, fill_value=np.nan)
print(prices)
prices[[0, 25, 60, -1]] = [80., 30., 75., 50.]
print(prices)
x = np.arange(len(prices))
print(x)
is_valid = ~np.isnan(prices)
print(is_valid)
prices = np.interp(x=x, xp=x[is_valid], fp=prices[is_valid])
print(prices)
prices += np.random.randn(len(prices)) * 2
print(prices)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
[80. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan 30. nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan 75. nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan 50.]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 

In [64]:
import numpy as np

#values to input
wells = list(plate.listChildren())

#values that will be calculated to simplify the reading of the np array
num_wells = len(wells)
num_fields = len(list(well.listChildren()))

#making the numpy array
dapi_imgs_np =

SyntaxError: invalid syntax (3719736511.py, line 11)

In [32]:
print(wells[0].listChildren())

<generator object BlitzObjectWrapper.listChildren at 0x00000150848BFC80>


In [42]:
wells = list(plate.listChildren())
print(len(wells[0].listChildren()))
image = wells[0].getImage(0)
pixels = image.getPrimaryPixels()
img_dapi = pixels.getPlane(0,0,0)

TypeError: object of type 'generator' has no len()

In [ ]:
import numpy as np

image_sizes = np.arra